# ASHviz: Getting Started

Sometime in 2017 I had done some data visualization experiments on ASH dump data I had obtained quite awhile ago. The data was from a 4-node RAC database running Oracle 10.2 under nontrivial load. So it's raw and real data.

I created a user group presentation using plots generated from the scripts but the process was painful and showing pure images without code context not all that instructive. I thought of just running the scripts in R Studio but that would have been extremely difficult to follow, albeit satisfying to see images generated in real time.

I had heard about iPython Notebooks and these seemed perfect for combining ability to run code and put context and discussion around it in formatted text. At the time converting the R to Python seemed like more work than I wanted to do just to start so I didn't. Learning that Jupyter Notebooks (successor to iPython) had an R kernel in addition to Python was enough to resurrect the project, as there definitely were interesting findings worth sharing there.

## Anaconda

One thing about programming in R and Python is the incredible amount of free and useful libraries and packages out there to use. There's tons of stuff and you want to be able to just use something you read about without going through a bunch of installation rigamarole first.

The open-source [Anaconda Distribution](https://www.anaconda.com) is a complete package management platform that manages dependencies and keeps versions updated for literally hundreds of packages and tools spanning multiple languages. Anything you want in R or Python is likely to be in there. Anaconda includes R Studio and Jupyter Notebooks. Recommended.

## Jupyter Notebooks

Upon starting Jupyter Notebooks for the first time I couldn't believe how simple yet effective the interface is. Basically, a notebook is a linear set of cells that are either code or markdown. Code cells are executed by an associated kernel process running an R interpreter, output is fed back and attached to the associated cell.

So my idea waa to convert each of the ASH dump visualization scripts into it's own notebook, separating the code into logical cells and adding markdown cells in between for explanation or motivation. 

## Setup.R

The original collection of R scripts included the first and critical script called `Setup.R` that read the specific ASH dump CSV files into an R data frame called `ashDF`. This data frame is the source for all plots.

In addition, `Setup.R` performed the following useful transforms on `ashDF`:

- `SAMPLE_TIME` has type `POSIXct` with microsecond granularity
- column `EST_COUNT` is added, computing the estimated event count using 1000ms sampling default and measured latency (`TIME_WAITED`)
- joined to a version of `V$EVENT_NAME` to get `EVENTNAME` and `WAITCLASS` by `EVENT_ID`
- column `STATE` is added with values: "CPU", "IO", or "WAIT"
- column 'STATE_CLASS' added with values: "CPU" or `WAITCLASS`
- column `MINIT` added (`SAMPLE_TIME` truncated) for simplified aggregation over time by minute
- all columns cast to factors except: `SAMPLE_ID`, `SAMPLE_TIME`, `MINIT`, `TIME_WAITED`, `WAIT_TIME`

So all the scripts began by sourcing `Setup.R` to obtain the nice data frame `ashDF` which they then proceed to manipulate and plot using `ggplot`.

## Notebook data sharing

The first obstacle to executing my plan was having the various notebooks all use the same `ashDF` data frame without all executing the `Setup.R` code every time. For one thing, unnecessary details about the source data (e.g. file names) could be exposed and for another why do complex data ingest and transform every time?

So I needed a good way to share the processed data frame between independently executing notebooks. They could each have their own copy, but it needs to be painless to import into the environment.

## R package `ashdat`

After considering alternatives, I decided to create an R package containing the processed data frame as that would allow for easy incorporation into Notebooks. This turned out to be quite a learning experience and took several days. Basically, an R package is a collection of R code and data objects organized and documented using a set of standards set by CRAN.

R Studio has built-in capabilities for developing and building R packages, and the `devtools` package from Hadley Wickham is very helpful. There are several good links on building R packages in the References cited below.

So I created an R package called `ashdat` that contains a data frame called `ASHDUMP1` which is the original data frame created by the `Setup.R` script. For now the package is located in the notebook directory and is installed and loaded by all the notebooks as follows:

In [11]:
# install ashdat from current directory
install.packages("./ashdat_0.1.0.tar.gz", repos = NULL)
library(ashdat)

## Initializing ashDF

Once the `ashdat` package is loaded we can initialize a local `ashDF` data frame using the `ASHDUMP1` data frame stored there as follows:

In [12]:
ashDF <- ashdat::ASHDUMP1
str(ashDF)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	214204 obs. of  36 variables:
 $ DBID                    : Factor w/ 1 level "3500743502": 1 1 1 1 1 1 1 1 1 1 ...
 $ INSTANCE_NUMBER         : Factor w/ 4 levels "1","2","3","4": 1 1 1 1 1 1 1 1 1 1 ...
 $ SAMPLE_ID               : int  35545015 35545015 35545015 35545015 35545015 35545015 35545015 35545015 35545015 35545015 ...
 $ SAMPLE_TIME             : POSIXct, format: "2006-10-09 12:19:24" "2006-10-09 12:19:24" ...
 $ SESSION_ID              : Factor w/ 620 levels "1522","1526",..: 22 39 75 83 92 111 120 137 163 234 ...
 $ SESSION_SERIAL.         : Factor w/ 1391 levels "1","3","4","5",..: 16 494 791 222 13 616 706 66 92 11 ...
 $ USER_ID                 : Factor w/ 6 levels "0","5","24","55",..: 4 4 4 4 4 4 4 4 4 4 ...
 $ SQL_ID                  : Factor w/ 1189 levels "","0486hsh0p6hcm",..: 380 243 238 222 243 222 380 222 111 243 ...
 $ SQL_CHILD_NUMBER        : Factor w/ 39 levels "0","1","2","3",..: 2 4 3 1 4 1 2 1 1 4 ...
 $ SQL_PLA

## End of the beginning

So at this point each of the original scripts should be convertible to Jupyter Notebooks in a relatively transparent way:

- import entire script into a single code cell
- subdivide that cell into logical pieces
- insert markdown cells for explanation and motivation

Mostly the scripts were simply sequences of plots, so having separate code cells for each plot seemed logical.

In the coming days and weeks I will be uploading the converted scripts as Jupyter Notebooks to my [ASHviz](https://github.com/jberesni/ASHviz/tree/master/Jupyter) github repository. The scripts are pretty raw and numerous failed experiements are included, but some commentary will be there. More interesting and/or useful results may be extracted into more focused and discursive articles, probably on LinkedIn to start.

## References

1. [Anaconda] (https://www.anaconda.com/)
2. [R Packages](http://r-pkgs.had.co.nz/) by Hadley Wickham
3. [Writing an R package from scratch](https://hilaryparker.com/2014/04/29/writing-an-r-package-from-scratch/) blog by Hilary Parker
4.[Making Your First R Package](http://tinyheero.github.io/jekyll/update/2015/07/26/making-your-first-R-package.html) blog by Fong Chun Chan
5. [R package primer] (https://kbroman.org/pkg_primer/) Karl Broman

Basically anyone getting started with using R for data analysis and plotting should become familiar with the incredibly useful software and documentation put out by Hadley Wickham.